# Topic Analysis

In [1]:
import datetime

import pandas as pd
import spacy
import re
import string
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

import nltk
from nltk.corpus import stopwords
from spacy.tokens import Token
from tqdm import tqdm


import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
from ipywidgets import interact

%matplotlib inline
np.random.seed(500)

In [2]:
df = pd.read_csv("..//data//Womens Clothing E-Commerce Reviews Sentiment v2.csv")

## Tokenizing the review text

In [3]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [4]:
def buildLDA(processed_docs, ip_num_topics=4):
    dictionary = gensim.corpora.Dictionary(processed_docs)

    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]

    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=ip_num_topics, id2word=dictionary, passes=2, workers=4, alpha =1)
    
    return dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf

In [5]:
def LDAtopicSummary(model, displaysubplot=[2,2]):
    for idx, topic in model.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))

In [6]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        #print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords',"Text"]
    return(sent_topics_df)


## LDA - Non-Recommendation 

### Non-Recommendation & Negative reviews

In [7]:
Neg_NR_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]<0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Neg_NR_processed_docs,8)
len(Neg_NR_processed_docs)

895

In [8]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=Neg_NR_processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.2677924873275451


In [9]:
#LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.003412 -0.004406       1        1  13.124423
5     -0.002859  0.001391       2        1  12.661332
2      0.000633  0.002299       3        1  12.652328
3      0.004203  0.000232       4        1  12.615535
1     -0.001638  0.003540       5        1  12.381181
6     -0.004983 -0.004353       6        1  12.309886
7      0.000358  0.000875       7        1  12.288750
0      0.000875  0.000423       8        1  11.966572, topic_info=    Category        Freq           Term       Total  loglift  logprob
81   Default  166.000000           size  166.000000  30.0000  30.0000
5    Default   99.000000           feel   99.000000  29.0000  29.0000
20   Default  172.000000           like  172.000000  28.0000  28.0000
12   Default   98.000000          tight   98.000000  27.0000  27.0000
138  Default   83.000000        picture   83.000000  26.0000  26.0000
3    Default  226.000000          dress  226.000000  25.0000  25.0000
75   Default   67.000000          chest   67.000000  24.0000  24.0000
30   Default  128.000000          color  128.000000  23.0000  23.0000
110  Default  140.000000          shirt  140.000000  22.0000  22.0000
144  Default   43.000000        stretch   43.000000  21.0000  21.0000
9    Default   79.000000         pretty   79.000000  20.0000  20.0000
83   Default   80.000000          waist   80.000000  19.0000  19.0000
52   Default   39.000000           item   39.000000  18.0000  18.0000
41   Default  147.000000           wear  147.000000  17.0000  17.0000
76   Default  105.000000           love  105.000000  16.0000  16.0000
158  Default   58.000000         blouse   58.000000  15.0000  15.0000
8    Default  131.000000          order  131.000000  14.0000  14.0000
47   Default  113.000000        quality  113.000000  13.0000  13.0000
182  Default   64.000000          awful   64.000000  12.0000  12.0000
40   Default   75.000000        usually   75.000000  11.0000  11.0000
32   Default   68.000000        flatter   68.000000  10.0000  10.0000
77   Default   52.000000           need   52.000000   9.0000   9.0000
44   Default   81.000000         little   81.000000   8.0000   8.0000
67   Default   32.000000       scratchy   32.000000   7.0000   7.0000
37   Default   77.000000          model   77.000000   6.0000   6.0000
2    Default  116.000000   disappointed  116.000000   5.0000   5.0000
111  Default   29.000000           true   29.000000   4.0000   4.0000
60   Default   92.000000  unfortunately   92.000000   3.0000   3.0000
11   Default  178.000000          small  178.000000   2.0000   2.0000
45   Default   60.000000           poor   60.000000   1.0000   1.0000
..       ...         ...            ...         ...      ...      ...
137   Topic8    6.046752        overall   41.819679   0.1892  -5.4177
191   Topic8    5.482210       horrible   38.098640   0.1844  -5.5157
173   Topic8    6.965743        pattern   48.796753   0.1764  -5.2762
110   Topic8   19.463346          shirt  140.823288   0.1441  -4.2486
171   Topic8    7.626053           high   54.271500   0.1606  -5.1856
12    Topic8   13.240363          tight   98.977692   0.1114  -4.6339
84    Topic8    9.715438          white   70.699196   0.1383  -4.9435
22    Topic8   17.800730         return  140.239517   0.0589  -4.3379
69    Topic8   14.307053        sweater  110.352631   0.0801  -4.5564
40    Topic8   10.222559        usually   75.826622   0.1192  -4.8926
108   Topic8   10.230103           cute   76.087738   0.1165  -4.8918
30    Topic8   16.001358          color  128.502502   0.0398  -4.4445
20    Topic8   20.499048           like  172.518753  -0.0071  -4.1968
2     Topic8   14.639079   disappointed  116.952736   0.0450  -4.5335
107   Topic8   10.545895           wide   79.972778   0.0971  -4.8614
46    Topic8    9.358191          price   69.241524   0.1217  -4.9809
41    Topic8   17.011370           wear  147.857346  -0.0

### Non-Recommendation & Positive reviews

In [10]:
Pos_NR_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]>0)].map(preprocess)
Pos_NR_processed_docs_org=df["Review Text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]>0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Pos_NR_processed_docs,4)

In [11]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=Pos_NR_processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.2680544876605632


In [12]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.005861 -0.002030       1        1  27.879185
2     -0.000571  0.002196       2        1  26.263851
1      0.004421 -0.004278       3        1  23.102503
3      0.002011  0.004112       4        1  22.754459, topic_info=    Category        Freq           Term       Total  loglift  logprob
49   Default  392.000000       material  392.000000  30.0000  30.0000
31   Default  499.000000           love  499.000000  29.0000  29.0000
284  Default  154.000000          print  154.000000  28.0000  28.0000
154  Default  421.000000          shirt  421.000000  27.0000  27.0000
45   Default  158.000000           jean  158.000000  26.0000  26.0000
15   Default  539.000000           size  539.000000  25.0000  25.0000
62   Default  333.000000        sweater  333.000000  24.0000  24.0000
278  Default  125.000000           lace  125.000000  23.0000  23.0000
2    Default  308.000000         design  308.000000  22.0000  22.0000
48   Default  594.000000           look  594.000000  21.0000  21.0000
14   Default  244.000000         petite  244.000000  20.0000  20.0000
25   Default  443.000000          color  443.000000  19.0000  19.0000
108  Default  260.000000         online  260.000000  18.0000  18.0000
106  Default  143.000000           neck  143.000000  17.0000  17.0000
283  Default  274.000000          model  274.000000  16.0000  16.0000
128  Default  263.000000        flatter  263.000000  15.0000  15.0000
313  Default   85.000000       straight   85.000000  14.0000  14.0000
96   Default  321.000000           good  321.000000  13.0000  13.0000
18   Default  219.000000          tight  219.000000  12.0000  12.0000
268  Default  118.000000           kind  118.000000  11.0000  11.0000
1    Default  168.000000    comfortable  168.000000  10.0000  10.0000
39   Default  207.000000        usually  207.000000   9.0000   9.0000
83   Default  198.000000          super  198.000000   8.0000   8.0000
229  Default  196.000000         expect  196.000000   7.0000   7.0000
247  Default  132.000000          black  132.000000   6.0000   6.0000
110  Default  219.000000  unfortunately  219.000000   5.0000   5.0000
104  Default  174.000000           time  174.000000   4.0000   4.0000
37   Default  244.000000          style  244.000000   3.0000   3.0000
3    Default  759.000000          dress  759.000000   2.0000   2.0000
188  Default   93.000000      perfectly   93.000000   1.0000   1.0000
..       ...         ...            ...         ...      ...      ...
20    Topic4   95.027779           want  353.820465   0.1658  -4.8810
108   Topic4   71.068878         online  260.307983   0.1822  -5.1715
139   Topic4   98.003036           cute  373.888855   0.1414  -4.8502
128   Topic4   71.203300        flatter  263.198547   0.1730  -5.1696
43    Topic4   90.805771           feel  349.055908   0.1339  -4.9264
283   Topic4   72.716438          model  274.712372   0.1513  -5.1486
16    Topic4  109.120346          small  433.522278   0.1009  -4.7427
94    Topic4   64.213791           soft  238.931641   0.1664  -5.2729
39    Topic4   56.559631        usually  207.392822   0.1811  -5.3999
60    Topic4   91.645149          short  369.230530   0.0869  -4.9172
40    Topic4  107.087181           wear  450.484436   0.0437  -4.7615
37    Topic4   63.434872          style  244.121826   0.1328  -5.2851
104   Topic4   47.890224           time  174.691666   0.1863  -5.5662
49    Topic4   93.092705       material  392.550598   0.0413  -4.9016
15    Topic4  121.184296           size  539.675842  -0.0132  -4.6378
25    Topic4  102.768623          color  443.106079   0.0191  -4.8027
12    Topic4   95.471588          order  405.123138   0.0350  -4.8763
3     Topic4  149.997833          dress  759.419922  -0.1415  -4.4245
46    Topic4   87.613487          large  386.398895  -0.0035  -4.9622
63    Topic4   83.000359          think  360.845978   0.0108  -

In [13]:
def demo(title):
    if not np.isnan(title):
        return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title-1].tolist()][:20])

df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Pos_NR_processed_docs_org) )
interact(demo, title=np.sort(df_dominant_topic["Dominant_Topic"].unique())+1);

interactive(children=(Dropdown(description='title', options=(1.0, 2.0, 3.0, 4.0), value=1.0), Output()), _dom_…

## LDA - Recommended Data

### Recommendation & Negative reviews

In [14]:
Neg_R_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]<0)].map(preprocess)
Neg_R_processed_docs_org=df["Review Text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]<0)].map(preprocess)

dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Neg_R_processed_docs,4)
len(Neg_R_processed_docs)

820

In [15]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=Neg_R_processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.29405933945400564


In [17]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.002086  0.003578       1        1  26.712090
3      0.001962  0.002136       2        1  25.224855
0     -0.006929 -0.000490       3        1  24.042837
2      0.002881 -0.005224       4        1  24.020218, topic_info=    Category        Freq        Term       Total  loglift  logprob
56   Default  110.000000      petite  110.000000  30.0000  30.0000
134  Default   77.000000       white   77.000000  29.0000  29.0000
48   Default   61.000000        wash   61.000000  28.0000  28.0000
23   Default   78.000000       skirt   78.000000  27.0000  27.0000
137  Default   42.000000        cozy   42.000000  26.0000  26.0000
109  Default   87.000000        jean   87.000000  25.0000  25.0000
129  Default   64.000000       comfy   64.000000  24.0000  24.0000
44   Default   87.000000    material   87.000000  23.0000  23.0000
4    Default  129.000000        love  129.000000  22.0000  22.0000
127  Default   61.000000        sale   61.000000  21.0000  21.0000
85   Default   32.000000      subtle   32.000000  20.0000  20.0000
7    Default  191.000000        size  191.000000  19.0000  19.0000
182  Default   31.000000      zipper   31.000000  18.0000  18.0000
66   Default   52.000000        base   52.000000  17.0000  17.0000
1    Default  225.000000       dress  225.000000  16.0000  16.0000
24   Default  208.000000       small  208.000000  15.0000  15.0000
60   Default   68.000000        pant   68.000000  14.0000  14.0000
83   Default   52.000000        pair   52.000000  13.0000  13.0000
14   Default  120.000000      fabric  120.000000  12.0000  12.0000
196  Default   52.000000        body   52.000000  11.0000  11.0000
21   Default   37.000000     problem   37.000000  10.0000  10.0000
74   Default   54.000000        line   54.000000   9.0000   9.0000
87   Default   53.000000   recommend   53.000000   8.0000   8.0000
25   Default   89.000000       waist   89.000000   7.0000   7.0000
50   Default  130.000000       color  130.000000   6.0000   6.0000
43   Default   33.000000        hole   33.000000   5.0000   5.0000
141  Default   55.000000       photo   55.000000   4.0000   4.0000
11   Default  108.000000     usually  108.000000   3.0000   3.0000
187  Default   35.000000       frame   35.000000   2.0000   2.0000
12   Default   62.000000  definitely   62.000000   1.0000   1.0000
..       ...         ...         ...         ...      ...      ...
179   Topic4    8.273710      shrink   29.079910   0.1693  -5.8234
7     Topic4   54.578545        size  191.884918   0.1690  -3.9368
10    Topic4   20.577703       usual   72.935532   0.1609  -4.9122
101   Topic4   24.680649       store   87.549934   0.1601  -4.7304
189   Topic4   11.070723     pattern   39.303413   0.1593  -5.5321
94    Topic4   18.107910       chest   64.528336   0.1555  -5.0401
14    Topic4   33.516518      fabric  120.896355   0.1434  -4.4244
50    Topic4   36.175961       color  130.848541   0.1406  -4.3480
106   Topic4   23.925911        soft   86.145844   0.1452  -4.7615
54    Topic4   30.481665        long  113.062721   0.1155  -4.5193
3     Topic4   41.553894      little  162.569366   0.0622  -4.2095
29    Topic4   27.954184       black  105.267128   0.1003  -4.6059
76    Topic4   20.501675    purchase   75.157173   0.1272  -4.9159
11    Topic4   28.002033     usually  108.119400   0.0753  -4.6042
110   Topic4   26.740910      medium  102.807083   0.0796  -4.6502
19    Topic4   40.330559        like  166.828064   0.0064  -4.2393
136   Topic4   29.851276      casual  117.158585   0.0590  -4.5402
38    Topic4   39.916924        wear  167.247528  -0.0064  -4.2496
16    Topic4   24.849375       large   96.817909   0.0663  -4.7236
61    Topic4   27.201309       tight  108.772881   0.0403  -4.6332
24    Topic4   44.728638       small  208.590027  -0.1135  -4.1358
5     Topic4   31.890451       order  136.969376  -0.0312  -4.4741
23    Topic4   

In [18]:
def demo(title):
    if not np.isnan(title):
        return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title-1].tolist()][:20])

df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Neg_R_processed_docs_org) )
interact(demo, title=np.sort(df_dominant_topic["Dominant_Topic"].unique())+1);

interactive(children=(Dropdown(description='title', options=(1.0, 2.0, 3.0, 4.0), value=1.0), Output()), _dom_…

### Recommendation & Positive reviews

In [22]:
Pos_R_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]>0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Pos_R_processed_docs,4)
len(Pos_R_processed_docs)

17605

In [23]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=Pos_R_processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.26608653578502295


In [24]:
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.001133  0.006178       1        1  27.033407
3     -0.005134 -0.004309       2        1  24.805363
0      0.006166 -0.002462       3        1  24.583019
1      0.000102  0.000593       4        1  23.578207, topic_info=    Category         Freq         Term        Total  loglift  logprob
12   Default  3104.000000         love  3104.000000  30.0000  30.0000
6    Default  3847.000000        dress  3847.000000  29.0000  29.0000
17   Default  1475.000000       pretty  1475.000000  28.0000  28.0000
211  Default  1814.000000      sweater  1814.000000  27.0000  27.0000
128  Default  1864.000000        large  1864.000000  26.0000  26.0000
83   Default  1322.000000         fall  1322.000000  25.0000  25.0000
43   Default  2587.000000         look  2587.000000  24.0000  24.0000
16   Default  1503.000000       petite  1503.000000  23.0000  23.0000
94   Default  1134.000000        right  1134.000000  22.0000  22.0000
178  Default  1880.000000         cute  1880.000000  21.0000  21.0000
39   Default  2538.000000        color  2538.000000  20.0000  20.0000
20   Default  1369.000000         true  1369.000000  19.0000  19.0000
14   Default  1001.000000       online  1001.000000  18.0000  18.0000
160  Default  1434.000000       medium  1434.000000  17.0000  17.0000
231  Default  2012.000000         soft  2012.000000  16.0000  16.0000
62   Default  2373.000000        small  2373.000000  15.0000  15.0000
184  Default   952.000000         need   952.000000  14.0000  14.0000
112  Default   601.000000         hope   601.000000  13.0000  13.0000
406  Default   672.000000         bust   672.000000  12.0000  12.0000
117  Default   805.000000        shape   805.000000  11.0000  11.0000
56   Default   929.000000        loose   929.000000  10.0000  10.0000
98   Default  1268.000000        black  1268.000000   9.0000   9.0000
19   Default  1196.000000        store  1196.000000   8.0000   8.0000
28   Default  2780.000000         wear  2780.000000   7.0000   7.0000
526  Default   680.000000        thing   680.000000   6.0000   6.0000
32   Default  1009.000000      legging  1009.000000   5.0000   5.0000
48   Default  3024.000000         size  3024.000000   4.0000   4.0000
173  Default   767.000000       unique   767.000000   3.0000   3.0000
7    Default   641.000000         glad   641.000000   2.0000   2.0000
171  Default  1217.000000        style  1217.000000   1.0000   1.0000
..       ...          ...          ...          ...      ...      ...
83    Topic4   384.062042         fall  1322.485718   0.2084  -5.3509
134   Topic4   459.407562        think  1615.101685   0.1876  -5.1718
380   Topic4   476.395966        short  1687.486084   0.1801  -5.1355
160   Topic4   404.807861       medium  1434.903687   0.1794  -5.2983
128   Topic4   510.388885        large  1864.364258   0.1493  -5.0666
129   Topic4   434.834869     material  1576.160522   0.1571  -5.2268
48    Topic4   768.841003         size  3024.203857   0.0753  -4.6569
191   Topic4   279.078522         blue   956.325195   0.2132  -5.6702
250   Topic4   338.877350      flatter  1194.335938   0.1851  -5.4761
147   Topic4   152.390793         coat   483.543640   0.2902  -6.2753
25    Topic4   716.754883        great  2882.410889   0.0532  -4.7270
62    Topic4   596.863037        small  2373.060547   0.0646  -4.9101
77    Topic4   423.454926         work  1594.815186   0.1188  -5.2533
12    Topic4   716.263184         love  3104.137207  -0.0216  -4.7277
6     Topic4   855.153748        dress  3847.148926  -0.0590  -4.5505
92    Topic4   360.197418      quality  1362.039673   0.1148  -5.4151
197   Topic4   457.103455    beautiful  1858.909912   0.0420  -5.1768
122   Topic4   300.818817         want  1115.263550   0.1345  -5.5952
151   Topic4   459.871521         jean  1887.196045   0.0329  -5.1708
34    Topic4   506.697540      perfect  2156.966797  -0.0037  -